In [58]:
from docplex.cp.model import *
from docplex.cp.config import get_default
import random

In [66]:
from config import setup
setup()

In [81]:
n = 40

F = []
H = []
for i in range(1,n+1):
    F.append('f'+str(i))
    H.append('h'+str(i))
print(F,H)

L= {}
for f in F:
    pref = list(H)
    random.shuffle(pref)
    L[f]=pref
for h in H:
    pref = list(F)
    random.shuffle(pref)
    L[h]=pref

#Lpos[h,f] postition de f dans L(h) + position de h dans L(h)
LPos = {}
for h in H:
    for f in F:
        LPos[h,f]=L[h].index(f)
        LPos[f,h]=L[f].index(h)

['f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40'] ['h1', 'h2', 'h3', 'h4', 'h5', 'h6', 'h7', 'h8', 'h9', 'h10', 'h11', 'h12', 'h13', 'h14', 'h15', 'h16', 'h17', 'h18', 'h19', 'h20', 'h21', 'h22', 'h23', 'h24', 'h25', 'h26', 'h27', 'h28', 'h29', 'h30', 'h31', 'h32', 'h33', 'h34', 'h35', 'h36', 'h37', 'h38', 'h39', 'h40']


In [82]:

mdl = CpoModel(name="Mariages")

In [83]:
#x[h,f] mariage entre h et f
x = {}
z = {}
for h in H:
    for f in F:
        x[h,f]=mdl.binary_var(name="x_" + h + "_" + f)
        z[h,f]=mdl.binary_var(name="z_" + h + "_" + f)

#y[i] cout du marriage de i (somme des couts du mariage pour les 2 partis) 
D = {}
for i in H+F:
    D[i]=mdl.integer_var(0,n-1,name="D_"+i)


#Ajout des contraintes
#Contrainte de monogamie
for h in H:
    mdl.add(sum([x[h,f] for f in F])==1)
for f in F:
    mdl.add(sum([x[h,f] for h in H])==1)


#Valeur de y (Position du partenaire dans le classement):
for h in H:
    mdl.add(D[h]==sum([(x[h,f]*LPos[h,f]) for f in F]))
for f in F:
    mdl.add(D[f]==sum([(x[h,f]*LPos[f,h]) for h in H]))

#Couple bloquant =
#si (h,f) est un couple bloquant si 
#   - le partenaire actuel de h et en plus haute postion de preference que f
#   - et le partenaire actuel de f et en plus haute postion de preference que h
for h in H:
    for f in F:
        mdl.add(mdl.if_then((D[h]>LPos[h,f]) & (D[f]>LPos[f,h]), z[h,f] == 1))
mdl.add(sum(z[h,f] for h in H for f in F)==0)


In [84]:
lsol = mdl.start_search(SearchType="DepthFirst",trace_log=False)

#Affichage des solutions:
for i, solution in enumerate(lsol):
    print("\n   SOLUTION "+str(i+1)+" :\n")
    for h in H:
        for f in F:
            if(solution[x[h,f]]==1):
                print("Mariage entre ",h," et ",f," (Couts = ",LPos[h,f],LPos[f,h],")")


In [ ]:
#Partie2
DH = mdl.integer_var(name="DH")
DF = mdl.integer_var(name="DF")

mdl.add(DH==max([D[h] for h in H]))
mdl.add(DF==max([D[f] for f in F]))
mdl.add(minimize(abs(DH-DF)))

solution = mdl.solve(trace_log=False)

print("\n   SOLUTION :\n")
for h in H:
    for f in F:
        if(solution[x[h,f]]==1):
            print("Mariage entre ",h," et ",f," (Couts = ",LPos[h,f],LPos[f,h],")")

print("\nDF = ",solution[DF])
print("DH = ",solution[DH])




   SOLUTION :

Mariage entre  h1  et  f11  (Couts =  1 0 )
Mariage entre  h2  et  f12  (Couts =  4 1 )
Mariage entre  h3  et  f16  (Couts =  0 1 )
Mariage entre  h4  et  f3  (Couts =  3 7 )
Mariage entre  h5  et  f14  (Couts =  2 5 )
Mariage entre  h6  et  f15  (Couts =  0 0 )
Mariage entre  h7  et  f6  (Couts =  2 4 )
Mariage entre  h8  et  f2  (Couts =  3 7 )
Mariage entre  h9  et  f18  (Couts =  9 8 )
Mariage entre  h10  et  f4  (Couts =  0 2 )
Mariage entre  h11  et  f13  (Couts =  3 1 )
Mariage entre  h12  et  f9  (Couts =  1 2 )
Mariage entre  h13  et  f20  (Couts =  2 6 )
Mariage entre  h14  et  f8  (Couts =  6 1 )
Mariage entre  h15  et  f10  (Couts =  0 5 )
Mariage entre  h16  et  f1  (Couts =  1 1 )
Mariage entre  h17  et  f7  (Couts =  7 4 )
Mariage entre  h18  et  f17  (Couts =  1 1 )
Mariage entre  h19  et  f19  (Couts =  0 2 )
Mariage entre  h20  et  f5  (Couts =  0 0 )

DF =  8
DH =  9
